In [16]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import selenium
import os
import pprint

import googlemaps

from collections import defaultdict

import time

import pickle

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
# Use fivethirtyeight styling, because I like it
import matplotlib.style as style
style.use('fivethirtyeight')

# Import private API key

MAPS_KEY = os.environ.get('MAPS_KEY')

# Change format of charts to .svg
%config InlineBackend.figure_format = 'svg'

In [20]:
%xmode

Exception reporting mode: Plain


In [21]:
# This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.
# Read the data into pandas dataframe
df = pd.read_csv('kc_house_data.csv')

In [22]:
# Let's take a look at the data
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [24]:
# Recast price column as integer, for simplicity
df['price'] = df.price.astype(int)

In [25]:
# Instantiate a Google Maps API session
gmaps = googlemaps.Client(key=MAPS_KEY)

In [26]:
coords = list(zip(df['lat'].astype(float), df['long'].astype(float)))

In [30]:
len(coords)

21613

In [28]:
# coords = list(zip(df2['lat'].astype(float), df2['long'].astype(float)))

NameError: name 'df2' is not defined

In [32]:
len(coords)

21613

In [31]:
formatted_addresses = []
json_addresses = []
neighborhoods_list = []
address_dictionary = defaultdict(str)

def get_addresses(coords_list):
    count = 0
    for coord_tuple in coords_list:
        count +=1
#         if count >= 1000:
#             time.sleep(120)
#             count = 0
        
        try:
            address = gmaps.reverse_geocode(coord_tuple)
            
            formatted_address = address[1]['formatted_address']
            formatted_addresses.append(formatted_address)
            json_addresses.append(address)
            
            neighborhood = address[0]['address_components'][2]['short_name']
            neighborhoods_list.append(neighborhood)
            
            address_dictionary[coord_tuple] = formatted_address
            
            print('Success')

        except:
            print('Could not get address.')
            formatted_addresses.append(None)

In [33]:
get_addresses(coords)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


limit_output extension: Maximum message size of 10000 exceeded with 10016 characters

In [34]:
len(formatted_addresses)

21613

In [39]:
# # Pickling original list so not to lose my work
# with open('address_list_final.pkl', 'wb') as f:
#     pickle.dump(list_of_addresses, f)

# with open('formatted_addresses.pkl', 'wb') as f:
#     pickle.dump(formatted_addresses, f)

# # Pickling like crazy because I don't want to lose my data.
# # Yes, I know pickling isn't always the best. Works for now.
# with open('json_addresses.pkl', 'wb') as f:
#     pickle.dump(json_addresses, f)

# with open('neighborhoods.pkl', 'wb') as f:
#     pickle.dump(neighborhoods_list, f)

# with open('address_dictionary.pkl', 'wb') as f:
#     pickle.dump(address_dictionary, f)

In [54]:
neighborhood_df.to_csv('neighborhood_csv.csv', index=False, header=False)

In [55]:
nh = pd.read_csv('neighborhood_csv.csv')

In [57]:
formatted_addresses_df = pd.DataFrame(formatted_addresses, columns=['Formatted Addresses'])

In [63]:
formatted_addresses_df.to_csv('formatted_addresses_csv.csv', index=False)

In [64]:
fa = pd.read_csv('formatted_addresses_csv.csv')

In [71]:
# Add formatted addresses from Google Maps API to our dataframe (!)
df['Address'] = formatted_addresses

In [76]:
# Adding neighborhood to dataframe
df['Neighborhood'] = neighborhoods_list

In [95]:
# Grab a list of all the zip codes from the formatted addresses list
# I love list comprehensions!
zip_list = [x.split(',')[-2][-5:] for x in formatted_addresses]

In [149]:
df['Imputed Zip Code'] = zip_list

In [154]:
def get_zips(x):
    if x == ' WA' or x == 'ngton':
        return 0
    else:
        return x

In [155]:
df['Imputed Zip Code'] = df['Imputed Zip Code'].apply(get_zips)

In [166]:
df['Imputed Zip Code'] = df['Imputed Zip Code'].astype(int)

In [191]:
# 205 Unique Neighborhoods. Should be sufficient for Walk Score/Transit Score/Bike Score etc. for our purposes.
df['Neighborhood'].nunique()

205

In [188]:
# grouped = df.groupby('Neighborhood')
df.sort_values('Neighborhood')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Address,Neighborhood,Imputed Zip Code
320,325059286,20140513T000000,819900,5,2.75,3150,7119,2.0,0,0,...,2013,0,98052,47.6759,-122.151,1560,8384,"8071 142nd Ave NE, Redmond, WA 98052, USA",142nd Ave NE,98052
6969,3037200010,20150305T000000,447500,2,2.25,1180,2090,2.0,0,0,...,2004,0,98122,47.6032,-122.310,1550,2812,"166 17th Ave, Seattle, WA 98122, USA",17th Ave,98122
61,9558200045,20140828T000000,289000,3,1.75,1260,8400,1.0,0,0,...,1954,0,98148,47.4366,-122.335,1290,8750,"18443 Occidental Ave S, Burien, WA 98148, USA",1st Ave S,98148
11744,1407300012,20150128T000000,400000,2,2.00,1050,1173,2.0,0,0,...,2004,0,98122,47.6181,-122.302,1340,1317,"1806 23rd Ave, Seattle, WA 98122, USA",23rd Ave,98122
14158,1773101020,20141020T000000,307000,5,1.50,1310,4800,1.5,0,0,...,1929,0,98106,47.5545,-122.365,960,4800,"5206 26th Ave S, Seattle, WA 98108, USA",26th Ave SW,98108
16511,1568100290,20140620T000000,337000,4,3.00,2240,8504,2.0,0,0,...,1992,0,98155,47.7348,-122.295,1570,8460,"14538 31st Ave NE, Shoreline, WA 98155, USA",31st Ave NE,98155
16661,9543000896,20140825T000000,237000,3,1.50,1800,9216,1.0,0,0,...,1950,0,98001,47.2739,-122.249,1400,10022,"Milwaukee Blvd S & 4th Ave S, Algona, WA 98001...",Algona,98001
7531,8856004328,20150327T000000,255000,4,2.50,2163,5882,2.0,0,0,...,2006,0,98001,47.2736,-122.251,1760,9600,"Milwaukee Blvd S & 4th Ave S, Algona, WA 98001...",Algona,98001
14423,9543000205,20150413T000000,139950,0,0.00,844,4269,1.0,0,0,...,1913,0,98001,47.2781,-122.250,1380,9600,"126 Milwaukee Blvd S, Algona, WA 98001, USA",Algona,98001
10569,8856004327,20140509T000000,248000,4,3.00,2163,5883,2.0,0,0,...,2006,0,98001,47.2734,-122.251,1700,10143,"417 Milwaukee Ave, Pacific, WA 98047, USA",Algona,98047


limit_output extension: Maximum message size of 10000 exceeded with 32353 characters